# TwitterAPI to explore Benford Laws
# GET FRIENDS/LIST & FOLLOWERS/LIST

## w210 Capstone
## Spring 2018

By: Vyas, Phat, Stacia, Kasey

Last update: 02/03/2018

## Set up environment

In [1]:
## Import twittersearch API
from TwitterAPI import TwitterAPI ## please `pip install TwitterAPI` if not yet installed 
#import TwitterSearch
import os
import pandas as pd
import json ## to load json format into dict
import time ## to extract date/time from retrieved data
from collections import defaultdict
#import datetime ## to get current date/time## Authentication

In [2]:
## twitter authentication

## --PHAT-- ##
#         consumer_key = 'X2SwXG1wIBLJiJRUunqIXyaNz',
#         consumer_secret = 'j0mtZ5BvcNE26rdRrySprP3iYhiAglUn8qtzGP5Rw4WXqm1lV8',
#         access_token = '41741727-9JoDDAAoTUo47z8Zo8uCQ1t5WLcpWnoQVSrImoB7q',
#         access_token_secret = 'dhEDyLezeg22qClu1qrIf3pVadN3n9KOxK8q72bCwvG42'

## --TED-- ##
consumer_key = "Vk9kCWPUw8KctdN4QGywZFpvd"
consumer_secret =  "CoPsToBCXDYWkaxoSnPf9qnNEPazxvHGDzexDzZ6KJxq9ZplQb"
access_token =  "820596649-FSomVy8OakTPYrCKrJQw2IfsUZXGk5Xp6kCzfbFt"
access_token_secret =  "c2jLc2HNQPKFGncaeNQzRonB4B9LOOKOU67lVcaOeEzYK"

## --RANDOM-- ##
# consumer_key = "vPgHzpN16ghYl2VQyoJogqoAR",
# consumer_secret =  "mNivOtVDdyK2tZFvlBm85inm1Fg1Lk0zmxmgX1aTZZPUenZKoy",
# access_token = "2506485501-CJtcW3B73XaoGHX5xzpz2Safw3Z9Mmi5L5Xakz7",
# access_token_secret = "o43l04V0mEAcOIwZ76D02f5LQhKLuZe40EpeuKj5f10du" 
api = TwitterAPI(consumer_key, consumer_secret, access_token, access_token_secret)

## Helper Functions

In [3]:
def date_time_transform(date_time):
    date_time_deconstruct = time.strptime(date_time, "%a %b %d %H:%M:%S +0000 %Y")
    date_output = str(date_time_deconstruct.tm_year) + '/' + str(date_time_deconstruct.tm_mon) + '/' + str(date_time_deconstruct.tm_mday)
    time_output = str(date_time_deconstruct.tm_hour) + ':' + str(date_time_deconstruct.tm_min)
    return date_output, time_output

def date_time_transform2(date_time):
    date_time_deconstruct = time.strptime(date_time, "%a %b %d %H:%M:%S %Y")
    date_output = str(date_time_deconstruct.tm_year) + str(date_time_deconstruct.tm_mon) + str(date_time_deconstruct.tm_mday)
    time_output = str(date_time_deconstruct.tm_hour) + str(date_time_deconstruct.tm_min)
    return date_output, time_output

## Load Varol-2017 dataset

In [4]:
## read varol-2017 file
file = open("../data/varol-2017.dat","r")
userid_list = []

for line in file:
    ## split userid with both identification
    line = line.strip()
    bot = line[len(line)-1]
    id = int(line[0:(len(line)-2)].strip())
    userid_list.append([id,bot])

file.close()

varol_userid_list = [row[0] for row in userid_list]

## create columns names
fields =  ['userid','bot']

## transform temp_list into data frame for exportation
df_output = pd.DataFrame(data = userid_list, columns = fields)

## get date/time for 
date_retrieved, time_retrieved = date_time_transform2(time.ctime())
file_name = 'varol-2017-formatted' + '.csv'

## separate by tab to prevent potential similar column split (i.e. comma, |)
df_output.to_csv(file_name,  sep = ',', index = False)

print('Successfully export file ', file_name)

Successfully export file  varol-2017-formatted.csv


## Call API to retrieve User details

Reference: https://developer.twitter.com/en/docs/accounts-and-users/follow-search-get-users/api-reference/get-users-show


*Multiple requests with rate-limit control*

In [5]:
def get_user_profile_multiple(userid_list):
    output = defaultdict(list)
    
    ## initiate rate_limit parameters
    ## control for rate limit of 5000 ids per request and 15 api calls per 15-minute window
    rate_limit_count = 1
    sleep_for =  900 ##-- sleep for 60 seconds
    count = 0
    
    for userid in userid_list:
        print("Row:",count)
        count += 1
        r = api.request('users/show',{'user_id':userid})
        try:
            output[userid].append(r.json())
            print("User id %s found",str(userid))
        except:
            print("Error for %s",str(userid))

        rate_limit_count += 1
        ## check for rate_limit of 15 api calls per 15-minute window
        if rate_limit_count > 900:
            rate_limit_count = 1
            time.sleep(sleep_for)
        print("--------------------------------------------------------------------")
    return output

In [49]:
varol_user_profiles = get_user_profile_multiple(varol_userid_list)

print(len(varol_user_profiles))

2573


In [48]:
with open('../data/varol_2017_user_profiles.txt', 'w') as outfile:
    json.dump(varol_user_profiles, outfile)

*Sample test*

In [32]:
userid_list = [3264569385,851583986270957568]

get_user_profile_multiple = get_user_profile_multiple(userid_list)
print(len(get_user_profile_multiple))

2


In [34]:
for key, id_list in get_user_profile_multiple.items():
    print(key)
    print(id_list)
    print("------------------------------------------------------")

3264569385
[{'id': 3264569385, 'id_str': '3264569385', 'name': 'Procopio Farfalameev', 'screen_name': 'lurinuneqofi', 'location': '', 'profile_location': None, 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 24, 'friends_count': 308, 'listed_count': 1, 'created_at': 'Sun May 17 01:26:16 +0000 2015', 'favourites_count': 1264, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 1267, 'lang': 'en', 'status': {'created_at': 'Fri Oct 02 20:50:16 +0000 2015', 'id': 650050188350984192, 'id_str': '650050188350984192', 'text': "RT @CometCoreInc: #COMET world's Floating Phone! Be the 1ST to put your hands on this #BEAUTY before it's gone: http://t.co/CUuFtlRww4 http…", 'truncated': False, 'entities': {'hashtags': [{'text': 'COMET', 'indices': [18, 24]}, {'text': 'BEAUTY', 'indices': [86, 93]}], 'symbols': [], 'user_mentions': [{'screen_name': 'CometCoreInc', 'name': 'Comet Core Inc', '

*One-off request*

In [4]:
def get_user_profile_single(user_id):
    r = api.request('users/show',{'user_id':user_id})
    return r.json()

In [7]:
user_id = 25073877 ## bot

test_ids = get_user_profile_single(user_id)
print(test_ids)

with open('djt.txt', 'w') as outfile:
    json.dump(test_ids, outfile)

{'id': 25073877, 'id_str': '25073877', 'name': 'Donald J. Trump', 'screen_name': 'realDonaldTrump', 'location': 'Washington, DC', 'profile_location': None, 'description': '45th President of the United States of America🇺🇸', 'url': 'https://t.co/OMxB0x7xC5', 'entities': {'url': {'urls': [{'url': 'https://t.co/OMxB0x7xC5', 'expanded_url': 'http://www.Instagram.com/realDonaldTrump', 'display_url': 'Instagram.com/realDonaldTrump', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 48760924, 'friends_count': 45, 'listed_count': 84595, 'created_at': 'Wed Mar 18 13:46:38 +0000 2009', 'favourites_count': 24, 'utc_offset': -18000, 'time_zone': 'Eastern Time (US & Canada)', 'geo_enabled': True, 'verified': True, 'statuses_count': 37087, 'lang': 'en', 'status': {'created_at': 'Fri Mar 09 01:08:14 +0000 2018', 'id': 971915531346436096, 'id_str': '971915531346436096', 'text': 'Kim Jong Un talked about denuclearization with the South Korean Representatives, no

## Call API to retrieve Follower/Friend details

Reference: https://developer.twitter.com/en/docs/accounts-and-users/follow-search-get-users/api-reference/get-followers-list


### Get list of followers/friends

*Multiple request with rate control limit*

In [15]:
def get_list_multiple(request_list_type, userid_list):
    ## initiate rate_limit parameters
    ## control for rate limit of 5000 ids per request and 15 api calls per 15-minute window
    rate_limit_count = 1
    sleep_for =  900 ##-- sleep for 60 seconds
    
    for row, userid in enumerate(userid_list):
        output = defaultdict(list)
        cursor = str(-1)
        print("Row:", row)
        print("User id:", str(userid))
        
        while cursor != 0:
            ## check for rate_limit of 15 api calls per 15-minute window
            if rate_limit_count > 15:
                rate_limit_count = 1
                time.sleep(sleep_for)
            
            r = api.request(request_list_type,{'user_id':userid, 'cursor':cursor, 'count':200})
            try:
                for user in r.json()['users']:
                    output[userid].append(user)
                cursor = r.json()['next_cursor']
            except:
                if "Rate limit exceeded" in r.text:
                    #print("rate limit exceed")
                    rate_limit_count = 1
                    time.sleep(sleep_for)
                else:
                    cursor = 0
                    print("User id %s errors!",str(userid))            
            
            rate_limit_count += 1
        if request_list_type == 'followers/list':
            file_name = '../data/varol_followers/user_' + str(userid) + '_' + str(date_retrieved) + '_' + str(time_retrieved) + '.txt'
        elif request_list_type == 'friends/list':
            file_name = '../data/varol_friends/user_' + str(userid) + '_' + str(date_retrieved) + '_' + str(time_retrieved) + '.txt'
        with open(file_name, 'w') as outfile:
            json.dump(output, outfile)
        print('Successfully export file ', file_name)
        print("--------------------------------------------------------------------")
    return output

In [ ]:
varol_friends_profiles = get_list_multiple('friends/list', varol_userid_list)
print(len(varol_followers_profiles))

Row: 0
User id: 3098421349
Successfully export file  ../data/varol_friends/user_3098421349_201826_628.txt
--------------------------------------------------------------------
Row: 1
User id: 554067867
Successfully export file  ../data/varol_friends/user_554067867_201826_628.txt
--------------------------------------------------------------------
Row: 2
User id: 256597786
Successfully export file  ../data/varol_friends/user_256597786_201826_628.txt
--------------------------------------------------------------------
Row: 3
User id: 103351486
Successfully export file  ../data/varol_friends/user_103351486_201826_628.txt
--------------------------------------------------------------------
Row: 4
User id: 199973582
Successfully export file  ../data/varol_friends/user_199973582_201826_628.txt
--------------------------------------------------------------------
Row: 5
User id: 22942001
Successfully export file  ../data/varol_friends/user_22942001_201826_628.txt
-------------------------------

Successfully export file  ../data/varol_friends/user_312931148_201826_628.txt
--------------------------------------------------------------------
Row: 47
User id: 961897154
Successfully export file  ../data/varol_friends/user_961897154_201826_628.txt
--------------------------------------------------------------------
Row: 48
User id: 2687223536
Successfully export file  ../data/varol_friends/user_2687223536_201826_628.txt
--------------------------------------------------------------------
Row: 49
User id: 154155992
Successfully export file  ../data/varol_friends/user_154155992_201826_628.txt
--------------------------------------------------------------------
Row: 50
User id: 258478116
Successfully export file  ../data/varol_friends/user_258478116_201826_628.txt
--------------------------------------------------------------------
Row: 51
User id: 2961735746
Successfully export file  ../data/varol_friends/user_2961735746_201826_628.txt
------------------------------------------------

Successfully export file  ../data/varol_friends/user_2231476190_201826_628.txt
--------------------------------------------------------------------
Row: 94
User id: 123214345
Successfully export file  ../data/varol_friends/user_123214345_201826_628.txt
--------------------------------------------------------------------
Row: 95
User id: 231718751
Successfully export file  ../data/varol_friends/user_231718751_201826_628.txt
--------------------------------------------------------------------
Row: 96
User id: 17861479
Successfully export file  ../data/varol_friends/user_17861479_201826_628.txt
--------------------------------------------------------------------
Row: 97
User id: 390146736
Successfully export file  ../data/varol_friends/user_390146736_201826_628.txt
--------------------------------------------------------------------
Row: 98
User id: 10086382
Successfully export file  ../data/varol_friends/user_10086382_201826_628.txt
-------------------------------------------------------

User id %s errors! 2754082609
Successfully export file  ../data/varol_friends/user_2754082609_201826_628.txt
--------------------------------------------------------------------
Row: 141
User id: 3032201618
User id %s errors! 3032201618
Successfully export file  ../data/varol_friends/user_3032201618_201826_628.txt
--------------------------------------------------------------------
Row: 142
User id: 3244584983
Successfully export file  ../data/varol_friends/user_3244584983_201826_628.txt
--------------------------------------------------------------------
Row: 143
User id: 463193055
Successfully export file  ../data/varol_friends/user_463193055_201826_628.txt
--------------------------------------------------------------------
Row: 144
User id: 386582662
Successfully export file  ../data/varol_friends/user_386582662_201826_628.txt
--------------------------------------------------------------------
Row: 145
User id: 3299614305
Successfully export file  ../data/varol_friends/user_329961

Successfully export file  ../data/varol_friends/user_2957206289_201826_628.txt
--------------------------------------------------------------------
Row: 187
User id: 90905432
Successfully export file  ../data/varol_friends/user_90905432_201826_628.txt
--------------------------------------------------------------------
Row: 188
User id: 190754647
Successfully export file  ../data/varol_friends/user_190754647_201826_628.txt
--------------------------------------------------------------------
Row: 189
User id: 2985092002
Successfully export file  ../data/varol_friends/user_2985092002_201826_628.txt
--------------------------------------------------------------------
Row: 190
User id: 3313759618
Successfully export file  ../data/varol_friends/user_3313759618_201826_628.txt
--------------------------------------------------------------------
Row: 191
User id: 2965318498
Successfully export file  ../data/varol_friends/user_2965318498_201826_628.txt
------------------------------------------

### Get followers/list

In [9]:
varol_followers_profiles = get_list_multiple('followers/list', varol_userid_list)
print(len(varol_followers_profiles))

Row: 0
User id: 3098421349
Successfully export file  ../data/varol_followers/user_3098421349_201826_628.txt
--------------------------------------------------------------------
Row: 1
User id: 554067867
Successfully export file  ../data/varol_followers/user_554067867_201826_628.txt
--------------------------------------------------------------------
Row: 2
User id: 256597786
Successfully export file  ../data/varol_followers/user_256597786_201826_628.txt
--------------------------------------------------------------------
Row: 3
User id: 103351486
Successfully export file  ../data/varol_followers/user_103351486_201826_628.txt
--------------------------------------------------------------------
Row: 4
User id: 199973582
Successfully export file  ../data/varol_followers/user_199973582_201826_628.txt
--------------------------------------------------------------------
Row: 5
User id: 22942001
Successfully export file  ../data/varol_followers/user_22942001_201826_628.txt
-------------------

TwitterConnectionError: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=5)

In [ ]:
## Get followers/list continue from the timeout above
varol_userid_list_next = varol_userid_list[20:]
varol_userid_list_next[0]
# varol_followers_profiles = get_list_multiple('followers/list', varol_userid_list)
# print(len(varol_followers_profiles))

*Sample Test*

In [ ]:
# request_list_type = 'friends/list'
userid_list = [3264569385]

test_list_multiple = get_list_multiple('friends/list', userid_list)
print(len(test_list_multiple))

Row: 0
User id: 3264569385


In [ ]:
for key, detail_list in test_list_multiple.items():
    print(key)
    print(detail_list[0])
    print("------------------")

*One-off request*

In [10]:
def get_list_single(request_list_type, user_id):
    cursor = str(-1)
    output = []
    sleep_for = 900 # sleep for 15 minutes
    
    ## control for rate limit of 200 users per request and 15 api calls per 15-minute window
    rate_limit_count = 1
    while cursor != 0:
        r = api.request(request_list_type,{'user_id':user_id, 'cursor':cursor, 'count':200})
        for user in r.json()['users']:
            output.append(user)

        cursor = r.json()['next_cursor']
        rate_limit_count += 1
        ## check for rate_limit of 15 api calls per 15-minute window
        if rate_limit_count > 15:
            rate_limit_count = 0
            time.sleep(sleep_for)
    return output

In [11]:
request_list_type = 'followers/list'
userid_list = 3264569385  

test_list_single = get_list_single(request_list_type, userid_list)
print(len(test_list_single))

24


In [14]:
for i in range(10):
    print(test_list_single[i])
    print("--------------------")

{'id': 3232190907, 'id_str': '3232190907', 'name': 'Amanda Vance', 'screen_name': 'AmandaVance253', 'location': '', 'description': 'Creative Director, Portrait Artist, Beer Enthusiast, Wife, Pet Parent: Rogue (Shih Tzu/Dalmatian), Tuesday (Great Dane), Bonnie & Clyde (2 Cats)', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 331, 'friends_count': 1401, 'listed_count': 20, 'created_at': 'Mon May 04 00:32:20 +0000 2015', 'favourites_count': 219, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 1323, 'lang': 'en', 'status': {'created_at': 'Mon Jan 29 15:49:10 +0000 2018', 'id': 958004096836960257, 'id_str': '958004096836960257', 'text': '27 fonts for absolutely FREE thanks to https://t.co/gbG31bIsLc!!! 😍🎁 Go grab yours! https://t.co/lK03xEY8eu via @TheHungryJPEG', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'thehungryjpeg', 'name': 'TheHungr

In [ ]:
r = api.request('friends/list',{'user_id':3264569385, 'cursor':-1, 'count':200})

print(r.json())

### Get ids of followers/friends

*One-off request*

In [58]:
def get_ids_single(request_ids_type, user_id):
    cursor = str(-1)
    output = []
    sleep_for =  900# sleep for 60 seconds
    
    ## control for rate limit of 5000 ids per request and 15 api calls per 15-minute window
    rate_limit_count = 1
    while cursor != 0:
        r = api.request(request_ids_type,{'user_id':user_id, 'cursor':cursor, 'count':5000})
        for user in r.json()['ids']:
            output.append(user)
        cursor = r.json()['next_cursor']
        rate_limit_count += 1
        ## check for rate_limit of 15 api calls per 15-minute window
        if rate_limit_count > 15:
            rate_limit_count = 0
            time.sleep(sleep_for)
    return output

In [59]:
request_ids_type = 'followers/ids'
user_id = 3264569385 ## bot

test_ids = get_ids_single(request_type, user_id)
print(len(test_ids))

24


*Multiple requests while controlling for rate-limit*

In [10]:
def get_ids_multiple(request_ids_type, userid_list):
    output = defaultdict(list)
    
    ## initiate rate_limit parameters
    ## control for rate limit of 5000 ids per request and 15 api calls per 15-minute window
    rate_limit_count = 1
    sleep_for =  900 ##-- sleep for 60 seconds
    
    for userid in userid_list:
        cursor = str(-1)
        while cursor != 0:
            r = api.request(request_ids_type,{'user_id':userid, 'cursor':cursor, 'count':5000})
            for user in r.json()['ids']:
                output[userid].append(user)
            cursor = r.json()['next_cursor']
            rate_limit_count += 1
            ## check for rate_limit of 15 api calls per 15-minute window
            if rate_limit_count > 15:
                rate_limit_count = 1
                time.sleep(sleep_for)
    return output

*Sample test*

In [11]:
request_ids_type = 'followers/ids'
userid_list = [3264569385,851583986270957568]

test_ids_multiple = get_ids_multiple(request_ids_type, userid_list)
print(len(test_ids_multiple))

2


In [16]:
for key, id_list in test_ids_multiple.items():
    print(key)
    print(id_list[0:10])
    print("------------------")

3264569385
[3232190907, 768798320009641984, 1161559692, 1191773365, 102099105, 4602275953, 25901558, 3038487852, 2906758607, 2632266395]
------------------
851583986270957568
[1674850602, 767548312970854400, 450405824, 837074857, 944377199305674753, 948554081454444545, 1864965229, 597346011, 4358164276, 191034919]
------------------


## Sandbox

In [ ]:
## pseudocode ##
"""
GET https://api.twitter.com/1.1/followers/list.json?cursor=-1&screen_name=twitterdev&skip_status=true&include_user_entities=false

cursor = -1

api_path = "https://api.twitter.com/1.1/endpoint.json?screen_name=targetUser"

do {
    url_with_cursor = api_path + "&cursor=" + cursor
    response_dictionary = perform_http_get_request_for_url( url_with_cursor )
    cursor = response_dictionary[ 'next_cursor' ]
}

while ( cursor != 0 )
"""

In [16]:
#cursor=-1&screen_name=twitterdev&skip_status=true&include_user_entities=false
request_type = "followers/ids"
# cursor = str(-1)
r = api.request('followers/ids',{'user_id':3264569385, 'cursor':-1})
print(r.text)

{"ids":[3232190907,768798320009641984,1161559692,1191773365,102099105,4602275953,25901558,3038487852,2906758607,2632266395,402944504,1964212543,217159694,839896928,1263011048,3279444135,2847572610,2665325118,3104359838,148687303,1116445639,3308023038,3012768110,3709697474],"next_cursor":0,"next_cursor_str":"0","previous_cursor":0,"previous_cursor_str":"0"}


In [25]:
#cursor=-1&screen_name=twitterdev&skip_status=true&include_user_entities=false
## Get followers/ids
request_type = "followers/ids"
cursor = str(-1)
followers_ids = []
while cursor != 0:
    r = api.request('followers/ids',{'user_id':3264569385, 'cursor':cursor})
#     url_with_cursor = api_path + "&cursor=" + cursor
#     response_dictionary = perform_http_get_request_for_url( url_with_cursor )
#     r_json = json.loads(r.text)
    followers_ids.append(r.json['ids'])
    cursor = r.json['next_cursor']
print(r.text)

{"ids":[3232190907,768798320009641984,1161559692,1191773365,102099105,4602275953,25901558,3038487852,2906758607,2632266395,402944504,1964212543,217159694,839896928,1263011048,3279444135,2847572610,2665325118,3104359838,148687303,1116445639,3308023038,3012768110,3709697474],"next_cursor":0,"next_cursor_str":"0","previous_cursor":0,"previous_cursor_str":"0"}


In [26]:
followers_ids

[[3232190907,
  768798320009641984,
  1161559692,
  1191773365,
  102099105,
  4602275953,
  25901558,
  3038487852,
  2906758607,
  2632266395,
  402944504,
  1964212543,
  217159694,
  839896928,
  1263011048,
  3279444135,
  2847572610,
  2665325118,
  3104359838,
  148687303,
  1116445639,
  3308023038,
  3012768110,
  3709697474]]

In [27]:
## Get friends/ids
cursor = str(-1)
friends_ids = []
while cursor != 0:
    r = api.request('friends/ids',{'user_id':3264569385, 'cursor':cursor})
#     url_with_cursor = api_path + "&cursor=" + cursor
#     response_dictionary = perform_http_get_request_for_url( url_with_cursor )
#     r_json = json.loads(r.text)
    friends_ids.append(r.json['ids'])
    cursor = r.json['next_cursor']
print(r.text)

{"ids":[2993877650,365778540,1321255268,130158957,3760981336,3545038575,3279444135,360218890,102099105,3803762962,2383915136,3194371338,330979922,173299423,3311308125,3431841855,1073552508,2807771343,3577464012,3161776604,106864455,2533811550,3038487852,839896928,477189543,2396354906,2784784744,2847572610,217159694,1964212543,2830161678,859776416,1263011048,3305889495,3346980087,3129219250,2189549689,65396961,365531306,14776527,19990183,14325066,42709904,199476462,711405654,79581485,24703428,18176129,209263909,243649223,275028216,907489496,1161559692,14683272,385647514,31233524,138830074,213860929,357175219,44471551,334662466,26792692,153846288,23972471,15763975,486282080,32005588,1397096156,22556102,27965474,2806905784,1069262498,3302481347,2788289665,244356158,52425453,2817737763,25298842,2485629211,2325897637,44041047,326429794,39868061,22669308,282272020,95252891,2716888059,27634215,22566780,2913961347,1649875777,554833436,705667663,2284188482,16644108,19206732,2700059544,43294996,

In [ ]:
friends_ids

In [ ]:
## Get friends/list
cursor = str(-1)
friends_list = []
sleep_for = 30 # sleep for 60 seconds
while cursor != 0:
    r = api.request('friends/list',{'user_id':3264569385, 'cursor':cursor})
#     url_with_cursor = api_path + "&cursor=" + cursor
#     response_dictionary = perform_http_get_request_for_url( url_with_cursor )
#    r_json = json.loads(r.text)

    try:
        for user in r.json()['users']:
            friends_list.append(user)
    #     friends_ids.append(r_json['ids'])
        cursor = r.json()['next_cursor']
        time.sleep(sleep_for)
    except:
        print("error!")
print(len(friends_list))
#print(r.text)

In [40]:
r.json()['errors']
r.status_code

429

In [ ]:
## Working with rate limite

# r = TwitterRestPager(api, 'search/tweets', {'q':'pizza', 'count':100})
# for item in r.get_iterator():
#     if 'text' in item:
#         print item['text']
#     elif 'message' in item and item['code'] == 88:
#         print 'SUSPEND, RATE LIMIT EXCEEDED: %s\n' % item['message']
#         break

In [ ]:
## Sample of requesting

# followers = set(id for id in api.request('followers/ids'))
# friends = set(id for id in api.request('friends/ids'))

# for id in unfollow:
#     r = api.request('friendships/destroy', {'user_id': id})
#     if r.status_code == 200:
#         status = r.json()
#         print 'unfollowed %s' % status['screen_name']

In [43]:
#cursor=-1&screen_name=twitterdev&skip_status=true&include_user_entities=false
## Get followers/ids
request_type = "followers/ids"
cursor = str(-1)
followers_ids = []
while cursor != 0:
    r = api.request('followers/ids',{'user_id':3264569385, 'cursor':cursor})
    followers_ids.append(r.json()['ids'])
    cursor = r.json()['next_cursor']
print(r.text)

{"ids":[3232190907,768798320009641984,1161559692,1191773365,102099105,4602275953,25901558,3038487852,2906758607,2632266395,402944504,1964212543,217159694,839896928,1263011048,3279444135,2847572610,2665325118,3104359838,148687303,1116445639,3308023038,3012768110,3709697474],"next_cursor":0,"next_cursor_str":"0","previous_cursor":0,"previous_cursor_str":"0"}


In [44]:
print(r.json()['next_cursor'])

0


In [48]:
request_ids_type = 'followers/ids'
user_id = 851583986270957568 ## bitfinexed

def get_ids_sb(request_ids_type, user_id):
    cursor = str(-1)
    output = []
    sleep_for =  900# sleep for 60 seconds
    
    ## control for rate limit of 5000 ids per request and 15 api calls per 15-minute window
    rate_limit_count = 0
    while cursor != 0:
        r = api.request(request_ids_type,{'user_id':user_id, 'cursor':cursor, 'count':5000})
        for user in r.json()['ids']:
            output.append(user)
        cursor = r.json()['next_cursor']
        rate_limit_count += 1
        ## check for rate_limit of 15 api calls per 15-minute window
        if rate_limit_count > 14:
            rate_limit_count = 0
            time.sleep(sleep_for)
    return output

test = get_ids_sb(request_type, user_id)
print(len(test))

37452


In [51]:
request_list_type = 'friends/list'
user_id = 851583986270957568

def get_list_sb(request_list_type, user_id):
    cursor = str(-1)
    output = []
    sleep_for = 900 # sleep for 60 seconds
    
    ## control for rate limit of 200 users per request and 15 api calls per 15-minute window
    rate_limit_count = 1
    while cursor != 0:
        r = api.request(request_list_type,{'user_id':user_id, 'cursor':cursor, 'count':200})
        for user in r.json()['users']:
            output.append(user)

        cursor = r.json()['next_cursor']
        rate_limit_count += 1
        ## check for rate_limit of 15 api calls per 15-minute window
        if rate_limit_count > 15:
            rate_limit_count = 0
            time.sleep(sleep_for)
    
    return output

bitfinexed_friends_list = get_list_sb(request_list_type, user_id)

In [53]:
len(bitfinexed_friends_list)

734

In [55]:
with open('bitfinexed.txt', 'w') as outfile:
    json.dump(bitfinexed_friends_list, outfile)